---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [342]:
import pandas as pd
import numpy as np

def blight_model():
    from sklearn.metrics import roc_curve, auc
    from sklearn.metrics import roc_auc_score
    from sklearn.model_selection import train_test_split
    from adspy_shared_utilities import plot_feature_importances
    from sklearn.ensemble import GradientBoostingClassifier
    from sklearn.feature_selection import SelectFromModel

    usecolumns1 = ['ticket_id', 'discount_amount', 'judgment_amount', 'fine_amount', 'late_fee', 'admin_fee', 'agency_name','compliance']
    usecolumns2 = ['ticket_id', 'discount_amount', 'judgment_amount', 'fine_amount', 'late_fee', 'admin_fee', 'agency_name']
    testcols = ['discount_amount', 'judgment_amount', 'fine_amount', 'late_fee', 'admin_fee', 'agency_name']
    
    dft = pd.read_csv('train.csv', encoding='iso-8859-1', low_memory=False, usecols=usecolumns1)
    
    #dfa = pd.read_csv('addresses.csv')
    #dfc = pd.read_csv('latlons.csv')
    #dfac = dfa.merge(dfc, left_on='address', right_on='address')
    dftest = pd.read_csv('test.csv', usecols=usecolumns2)

    #df = dft.merge(dfac, left_on='ticket_id', right_on='ticket_id')
    df = dft
    
    dft.dropna(subset=['compliance'], inplace=True)
    
    
    TX = dft[usecolumns2].copy()
    TX_test = dftest.copy()
    TX = pd.get_dummies(TX, columns=['agency_name'])
    TX_test = pd.get_dummies(TX_test, columns=['agency_name'])
    TX_test = TX_test.reindex(columns=TX.columns, fill_value=0)
    
    y = dft['compliance'].astype('category')
    
    t2col = list(TX.columns)
    t2col.remove('ticket_id')
    
    X_train, X_test, y_train, y_test = train_test_split(TX, y, random_state=0)
   

    clf = GradientBoostingClassifier(n_estimators=300, learning_rate=0.05,
             max_depth=4, random_state=0).fit(X_train[t2col], y_train)
    
    
    y_pred = clf.predict_proba(X_test[t2col])
    

    res_list = [yp[1] for yp in y_pred] #return result of the positive class (second column)
    answer = pd.Series(res_list, index=X_test.ticket_id)
    
    print('ROCAUC score', roc_auc_score(y_test, answer))
    # Create a selector object that will use the random forest classifier to identify
    # features that have an importance of more than 0.15
    
    feat_labels = t2col
    # Print the name and gini importance of each feature
    for feature in zip(feat_labels, clf.feature_importances_):
        print(feature)
    
    sfm = SelectFromModel(clf, threshold=0.15)

    # Train the selector
    sfm.fit(X_train, y_train)
    # Print the names of the most important features
    for feature_list_index in sfm.get_support(indices=True):
        print(feat_labels[feature_list_index])

    y_pred_test = clf.predict_proba(TX_test[t2col])
    res_list_test=[yp[1] for yp in y_pred_test]
    test_answer=pd.Series(res_list_test, index=TX_test.ticket_id)
    
        
    return test_answer





In [343]:
blight_model()

ROCAUC score 0.7573900786477045
('discount_amount', 0.2134051721432804)
('judgment_amount', 0.04443387772244397)
('fine_amount', 0.02736489753303754)
('late_fee', 0.7035195914114081)
('admin_fee', 0.0)
('agency_name_Buildings, Safety Engineering & Env Department', 0.0016810816498140364)
('agency_name_Department of Public Works', 0.0006129088013723852)
('agency_name_Detroit Police Department', 0.0026796806388724673)
('agency_name_Health Department', 0.006302573240474386)
('agency_name_Neighborhood City Halls', 2.1685929685224054e-07)
judgment_amount
admin_fee


ticket_id
284932    0.063536
285362    0.024771
285361    0.073927
285338    0.063536
285346    0.073927
            ...   
376496    0.005424
376497    0.005424
376499    0.059639
376500    0.059639
369851    0.271245
Length: 61001, dtype: float64